In [ ]:
using Pkg; Pkg.activate(".")
using SparseArrays, QuadGK, ForwardDiff, LinearAlgebra, LaTeXStrings, CairoMakie
include("tools.jl")

## Motivation for Spectral Methods

This is a short notebook to motivate the study of spectral methods for solving PDEs, especially in contrast with finite difference and finite element methods. 

We consider a boundary value problem with periodic boundary conditions: 

$$
\begin{aligned}
  - u'' + u &= f, \quad x \in (-\pi, \pi), \\ 
  u(-\pi) &= u(\pi), \\ 
  u'(-\pi) &= u'(\pi)
\end{aligned}
$$

We added the mass term so that there is no subtletly about existence and uniquess; without it we would require that $\int f = 0$ and add another condition $\int u = 0$ but that is the only change required. 


The standard second-order central finite difference scheme (equivalent to P1-FEM) is given by 

$$ 
   - \frac{U_{n+1} - 2 U_n + U_{n-1}}{h^2} + U_n = F_n, \quad n = 1, \dots, 2N;
$$

where $h = \pi/N, x_n = -\pi + n h, F_n = f(x_n)$ and $U_n \approx u(x_n)$. The PBC is implemented via the identification $U_0 = U_{2N}$ and $U_1 = U_{2N+1}$.

The next cell gives a simple implementation of this scheme. 

In [ ]:
#
# A simple finite difference / FEM solver
# for the periodic boundary value problem above
#

function fd_method(f_fun, N)
    h = pi/N
    X = range(-pi, pi, 2*N+1)
    A = zeros(2*N, 2*N)
    F = zeros(2*N)
    for n = 2:(2*N)
        A[n,n] = 2/h^2 + 1 
        A[n-1, n] = -1/h^2
        A[n, n-1] = -1/h^2
        F[n] = f_fun(X[n+1])
    end 
    A[1,1] = 2/h^2 + 1 
    A[1, 2*N] = -1/h^2 
    A[2*N, 1] = -1/h^2
    F[1] = f_fun(X[2])
    
    _U = sparse(A) \ F 
    U = [ [_U[end]]; _U]
    return X, U 
end

In [ ]:
# pick some random periodic applied force
f_fun = x -> 1 / (2+sin(x))
# f_fun = x -> 1 / (1+x^2)

X, U = fd_method(f_fun, 8)

# plot the solution
fig1 = Figure(size = (400, 250)); ax1 = Axis(fig1[1,1])
scatterlines!(X, U, label = "finite-diff"); fig1

By contrast the spectral method works in a representation where the differential operator diagonalizes. Expanding in a Fourier series, 

$$
   u(x) = \sum_{k \in \mathbb{Z}} \hat{u}_k e^{i k x} 
$$

one can quickly derive that 

$$
   \hat{u}_k = \frac{\hat{f}_k}{1 + k^2}
$$

Then we truncate to keep only the terms up to $|k| \leq N$ and this gives the approximate solution 

$$
  u_N(x) = {\rm Re}\bigg( \sum_{k = -N}^N \frac{\hat{f}_k}{1 + k^2} e^{i k x} \bigg).
$$

Taking the real part is technically not required here, but in general good practice since in some situations truncation can lead to spurious complex components. 

In [ ]:
# WARNING: 
# This code is only for motivation! Do NOT solve real problems this way!
# We will learn how to implement this "correctly" later.

function naive_spectral(f_fun, N)
    uhat = zeros(ComplexF64, 2*N+1)
    for k = -N:N 
        g = x -> f_fun(x) * exp(-im * k * x)
        fhat_k = quadgk(g, -pi, pi; rtol = 1e-6, atol=1e-6)[1] / (2*pi)
        uhat[k+N+1] = fhat_k / (1 + k^2)
    end
    return uhat 
end

function naive_eval_trig(x, uhat) 
    N = (length(uhat)-1) ÷ 2
    return real( sum(uhat[k+N+1] * exp(im * x * k) for k = -N:N) )
end



In [ ]:
Û = naive_spectral(f_fun, 8)

xp = range(-pi, pi, 100)
up = naive_eval_trig.(xp, Ref(Û))

fig2 = Figure(size = (500, 250))
ax2 = Axis(fig2[1,1], xlabel = "x", ylabel = "u(x)")
scatterlines!(ax2, X, U, label = "finite-diff")
lines!(ax2, xp, up, color = :red, label = "spectral")
Legend(fig2[1, 2], ax2)
fig2

To see the real power of spectral methods, let us perform an empirical error analysis via the method of manufactured solutions.

In [ ]:
# define a known exact solution and compute the resulting f
u_ex = x -> 1 / (1 + cos(x)^2)
# u_ex = x -> abs(sin(x))^3  
# u_ex = x -> exp(cos(x))
du_ex = x -> ForwardDiff.derivative(u_ex, x)
f_fun = x -> u_ex(x) - ForwardDiff.derivative(du_ex, x)

# fine grid on which to evaluate the errors
xe = range(-pi, pi, length=1000)
ue = u_ex.(xe)

# looking at errors for the finite-difference scheme
NNfd = [4, 8, 16, 32, 64, 128]
err_fd = zeros(length(NNfd))
for (i, N) in enumerate(NNfd) 
    X, U = fd_method(f_fun, N)
    ufd = eval_p1.(xe, Ref(U))  # this is implemented in tools.jl
    err_fd[i] = norm(ue - ufd, Inf)
end

# looking at errors for the spectral method
NNspec = [4, 8, 12, 16, 20, 24, 28]
err_spec = zeros(length(NNspec))
for (i, N) in enumerate(NNspec)     
    Uhat = naive_spectral(f_fun, N)
    uspec = naive_eval_trig.(xe, Ref(Uhat))
    err_spec[i] = norm(ue - uspec, Inf)
end


In [ ]:
fig3 = Figure(size = (500, 300))
ax3 = Axis(fig3[1,1], xlabel = L"N", ylabel = L"|\!|u - u_N|\!|_\infty", 
           xscale = log10, yscale = log10)
scatterlines!(ax3, NNfd, err_fd, label = "finite-diff")
scatterlines!(ax3, NNspec, err_spec, label = "spectral")
lines!(ax3, [10, 100], 0.3*[10, 100].^(-1), color = :black, label = L"\sim N^{-1}", linestyle = :dash)
Legend(fig3[1, 2], ax3)
fig3

To see the rate for the spectral method we need to switch to a linear scale.

In [ ]:
fig4 = Figure(size = (500, 300))
ax4 = Axis(fig4[1,1], xlabel = L"N", ylabel = L"|\!|u - u_N|\!|_\infty", 
           xscale = identity, yscale = log10)
xlims!(ax4, (0, NNspec[end]+5))
scatterlines!(ax4, NNfd, err_fd, label = "finite-diff")
scatterlines!(ax4, NNspec, err_spec, label = "spectral")
lines!(ax4, [15, 28], exp.(- 0.87 * [15, 28]), color = :black, 
       label = L"\sim \exp(- \alpha N)", linestyle = :dash)
Legend(fig4[1, 2], ax4)
fig4

We get an exponential rate of convergence. We will study how this is achieved, and then develop efficient and numerically stable algorithms that allow us to exploit such fast convergence rates very effectively.